In [ ]:
OTTENIMENTO DATA NASCITA ARTISTI OTTIMIZZATO

In [3]:
import pandas as pd
import wikipedia
import re
import concurrent.futures
from tqdm import tqdm

wikipedia.set_lang("it")
wikipedia.set_rate_limiting(True)

file_path = "File Unico Sanremo.xlsx"
df = pd.read_excel(file_path, sheet_name="Artisti Unici con genere")

# Filtra solo Maschio e Femmina
df = df[df["Genere"].isin(["Maschio", "Femmina"])].copy()

dob_cache = {}

mesi_ita = {
    "gennaio": "01", "febbraio": "02", "marzo": "03", "aprile": "04",
    "maggio": "05", "giugno": "06", "luglio": "07", "agosto": "08",
    "settembre": "09", "ottobre": "10", "novembre": "11", "dicembre": "12"
}

# Tentativi alternativi per cercare la pagina Wikipedia giusta
def cerca_pagina(artista):
    varianti = [artista]
    parole_chiave = ["cantante", "musicista", "cantautore", "cantautrice", "sanremo"]

    # Aggiungi varianti con parole chiave
    for parola in parole_chiave:
        varianti.append(f"{artista} {parola}")

    for nome in varianti:
        try:
            return wikipedia.page(nome, auto_suggest=False, preload=False)
        except wikipedia.exceptions.DisambiguationError as e:
            continue  # Prova il prossimo
        except wikipedia.exceptions.PageError:
            continue
        except Exception:
            continue

    return None

def estrai_data_nascita(artista):
    if artista in dob_cache:
        return dob_cache[artista]

    if any(sep in artista.lower() for sep in [",", " e ", " - "]):
        print(f"⚠️ Nomi multipli rilevati, salto: {artista}")
        return "Da verificare"

    print(f"🟨 Estrazione data per: {artista}")
    pagina = cerca_pagina(artista)

    if pagina is None:
        print(f"❌ Nessuna pagina trovata per: {artista}")
        return "Da verificare"

    testo = (pagina.summary + " " + pagina.content).lower()[:1000]

    match = re.search(r'(\d{1,2})\s+([a-z]+)\s+(\d{4})', testo)
    if match:
        giorno = match.group(1).zfill(2)
        mese_testo = match.group(2)
        mese = mesi_ita.get(mese_testo)
        anno = match.group(3)

        if mese:
            data = f"{giorno}/{mese}/{anno}"
            dob_cache[artista] = data
            return data

    print(f"⚠️ Nessuna data trovata per: {artista}")
    return "Da verificare"

def apply_parallel(df, func, descrizione):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(func, artista): artista for artista in df["Artista"]}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=descrizione):
            artista = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"❌ Errore durante {descrizione} su: {artista} — {e}")
                results.append("Errore")
    return results

df["Data di nascita"] = apply_parallel(df, estrai_data_nascita, "Estrazione ottimizzata")
output_file = "Artisti_unici_filtro_genere_e_ottimizzati.xlsx"
df.to_excel(output_file, index=False)

print(f"\n✅ File salvato con successo: {output_file}")


🟨 Estrazione data per: Achille Togliani
🟨 Estrazione data per: Oscar Carboni
🟨 Estrazione data per: Gino Latilla
🟨 Estrazione data per: Zucchero
🟨 Estrazione data per: Will
🟨 Estrazione data per: Virginio
🟨 Estrazione data per: Ultimo
🟨 Estrazione data per: Tricarico
🟨 Estrazione data per: Tiziano Orecchio
🟨 Estrazione data per: Stefano Tosi


Estrazione ottimizzata:   0%|          | 0/470 [00:00<?, ?it/s]D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Estrazione ottimizzata:   1%|          | 4/470 [00:01<02:28,  3.14it/s]

🟨 Estrazione data per: Ugo Molinari
⚠️ Nessuna data trovata per: Virginio
🟨 Estrazione data per: Gianni Marzocchi
⚠️ Nessuna data trovata per: Tricarico
🟨 Estrazione data per: Stefano Rosso
🟨 Estrazione data per: Silvano Vittorio
🟨 Estrazione data per: Sangiovanni


Estrazione ottimizzata:   1%|▏         | 6/470 [00:02<02:12,  3.50it/s]

⚠️ Nessuna data trovata per: Zucchero
🟨 Estrazione data per: Samuel


...

❌ Nessuna pagina trovata per: Angelica


Estrazione ottimizzata: 100%|█████████▉| 469/470 [01:26<00:00,  2.34it/s]

❌ Nessuna pagina trovata per: Alice
❌ Nessuna pagina trovata per: Gaia


Estrazione ottimizzata: 100%|██████████| 470/470 [01:26<00:00,  5.42it/s]

❌ Nessuna pagina trovata per: Shari

✅ File salvato con successo: Artisti_unici_filtro_genere_e_ottimizzati.xlsx
